<a href="https://colab.research.google.com/github/hardik1408/MADHAV-LABS/blob/main/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://zenodo.org/records/3685367/files/TinySOL.tar.gz
!wget https://zenodo.org/records/3685367/files/TinySOL_metadata.csv


--2024-02-06 05:47:31--  https://zenodo.org/records/3685367/files/TinySOL.tar.gz
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.185.79.172, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1026917185 (979M) [application/octet-stream]
Saving to: ‘TinySOL.tar.gz’

TinySOL.tar.gz      100%[===================>] 979.34M  28.8MB/s    in 37s     

2024-02-06 05:48:10 (26.3 MB/s) - ‘TinySOL.tar.gz’ saved [1026917185/1026917185]

--2024-02-06 05:48:10--  https://zenodo.org/records/3685367/files/TinySOL_metadata.csv
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.184.98.238, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 317576 (310K) [text/plain]
Saving to: ‘TinySOL_metadata.csv’

TinySOL_metadata.cs 100%[===================>] 310.13K   794KB/s    in 0.4s    

2024-02-06 05:48:

In [3]:
import tarfile

In [11]:

!pip install librosa

In [12]:

import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import numpy as np
import IPython.display as ipd
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import sklearn

In [4]:
file_path = '/content/TinySOL.tar.gz'

# Specify the destination directory where you want to extract the contents
extract_path = '/content'

# Open the tar.gz file and extract its contents
with tarfile.open(file_path, 'r:gz') as tar:
    tar.extractall(extract_path)

In [5]:
import pandas as pd

# Replace 'your_file.csv' with the actual name of your CSV file
file_path = '/content/TinySOL_metadata.csv'

# Load the CSV file into a Pandas DataFrame
df = pd.read_csv(file_path)

In [6]:
df.head()

,Path,Fold,Family,Instrument (abbr.),Instrument (in full),Technique (abbr.),Technique (in full),Pitch,Pitch ID,Dynamics,Dynamics ID,Instance ID,String ID (if applicable),Needed digital retuning
0,Brass/Bass_Tuba/ordinario/BTb-ord-F#1-pp-N-N.wav,2,Brass,BTb,Bass Tuba,ord,ordinario,F#1,30,pp,0,0,NaN,False
1,Brass/Bass_Tuba/ordinario/BTb-ord-G1-pp-N-R100...,4,Brass,BTb,Bass Tuba,ord,ordinario,G1,31,pp,0,0,NaN,True
2,Brass/Bass_Tuba/ordinario/BTb-ord-G#1-pp-N-T16...,3,Brass,BTb,Bass Tuba,ord,ordinario,G#1,32,pp,0,0,NaN,True
3,Brass/Bass_Tuba/ordinario/BTb-ord-A1-pp-N-T23d...,2,Brass,BTb,Bass Tuba,ord,ordinario,A1,33,pp,0,0,NaN,True
4,Brass/Bass_Tuba/ordinario/BTb-ord-A#1-pp-N-N.wav,0,Brass,BTb,Bass Tuba,ord,ordinario,A#1,34,pp,0,0,NaN,False


In [13]:
len(df['Path'])

2913

In [10]:
def mfccs_features_extractor(file):
  audio, sample_rate = librosa.load(file)
  mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate,n_mfcc= 40);
  mfccs_scaled_features = np.mean(mfccs.T, axis=0)

  return mfccs_scaled_features

In [14]:
# List to store extracted features
extracted_features = []
inst_name= []
# Loop through each file
for i in range(2913):
  inst_name.append(df['Instrument (in full)'][i])
  file_name= '/content/' + df['Path'][i]
  extracted_features.append(mfccs_features_extractor(file_name))

In [16]:
features_df = pd.DataFrame({'class': inst_name, 'features': extracted_features})

In [17]:
X=np.array(features_df['features'].tolist())
y=np.array(features_df['class'].tolist())
le=LabelEncoder()
ti=le.fit_transform(y)
y=to_categorical(ti)

In [18]:
# Encoding the classes

originallabels=le.classes_
print("Original labels: ", originallabels)
print("Encoded labels: ", np.unique(ti))

Original labels:  ['Accordion' 'Alto Saxophone' 'Bass Tuba' 'Bassoon' 'Cello'
 'Clarinet in Bb' 'Contrabass' 'Flute' 'French Horn' 'Oboe' 'Trombone'
 'Trumpet in C' 'Viola' 'Violin']
Encoded labels:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]


In [19]:
# Splitting the dataset into train and test

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=127)


In [20]:
X_train_flat = np.array([x.flatten() for x in X_train])
X_test_flat = np.array([x.flatten() for x in X_train])

In [30]:
X_train_flat

array([[-549.4563   ,    5.0285707,  -13.947794 , ...,   -5.318942 ,
          17.268082 ,   38.473454 ],
       [-560.7889   ,   86.058495 ,   -7.537057 , ...,    6.315191 ,
          -2.937526 ,   -9.077227 ],
       [-735.6785   ,   43.11193  ,  -12.538934 , ...,  -17.094145 ,
          21.44201  ,   10.351549 ],
       ...,
       [-733.28     ,   12.70858  ,   -5.497362 , ...,   -1.2634788,
          24.121836 ,   -1.5230322],
       [-691.7369   ,  159.53209  ,   80.08078  , ...,   47.177986 ,
          32.845997 ,   12.9170065],
       [-564.0639   ,  258.73044  ,   29.321585 , ...,   -1.2253263,
          -2.3293302,   -2.0929234]], dtype=float32)

In [32]:
from tensorflow.keras import layers,models
model = models.Sequential()
model.add(layers.Conv1D(32, kernel_size=5, activation='relu', input_shape=(X_train_flat.shape[1], 1)))
model.add(layers.MaxPooling1D(pool_size=1))
model.add(layers.Conv1D(64, kernel_size=5, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=1))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(14, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [33]:

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 36, 32)            192       
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 36, 32)            0         
 g1D)                                                            
                                                                 
 conv1d_3 (Conv1D)           (None, 32, 64)            10304     
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 32, 64)            0         
 g1D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 128)              

In [34]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='Adam')

In [36]:
from keras.callbacks import ModelCheckpoint
from datetime import datetime
num_epochs = 30
num_batch_size = 4
checkpointer = ModelCheckpoint(filepath='saved_model.hdf5',verbose=1,save_best_only=True)
start = datetime.now()
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test),  verbose=1,callbacks=[checkpointer])
duration = datetime.now() - start
print("Training completed in time: ", duration)


Epoch 1/30
583/583 [==============================] - ETA: 0s - loss: 0.5288 - accuracy: 0.8275
Epoch 1: val_loss improved from inf to 0.19353, saving model to saved_model.hdf5
583/583 [==============================] - 5s 9ms/step - loss: 0.5288 - accuracy: 0.8275 - val_loss: 0.1935 - val_accuracy: 0.9348
Epoch 2/30
 18/583 [..............................] - ETA: 3s - loss: 0.2400 - accuracy: 0.9306

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


576/583 [============================>.] - ETA: 0s - loss: 0.3320 - accuracy: 0.8824
Epoch 2: val_loss improved from 0.19353 to 0.14210, saving model to saved_model.hdf5
583/583 [==============================] - 4s 6ms/step - loss: 0.3330 - accuracy: 0.8820 - val_loss: 0.1421 - val_accuracy: 0.9503
Epoch 3/30
581/583 [============================>.] - ETA: 0s - loss: 0.2436 - accuracy: 0.9204
Epoch 3: val_loss improved from 0.14210 to 0.11453, saving model to saved_model.hdf5
583/583 [==============================] - 4s 6ms/step - loss: 0.2430 - accuracy: 0.9206 - val_loss: 0.1145 - val_accuracy: 0.9640
Epoch 4/30
579/583 [============================>.] - ETA: 0s - loss: 0.1884 - accuracy: 0.9361
Epoch 4: val_loss improved from 0.11453 to 0.11175, saving model to saved_model.hdf5
583/583 [==============================] - 5s 8ms/step - loss: 0.1891 - accuracy: 0.9361 - val_loss: 0.1117 - val_accuracy: 0.9691
Epoch 5/30
581/583 [============================>.] - ETA: 0s - loss: 0.163

In [37]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(f"test set accuracy: {test_accuracy[1] * 100}%")

train_accuracy=model.evaluate(X_train,y_train,verbose=0)
print(f"train set accuracy: {train_accuracy[1] * 100}%")

test set accuracy: 98.62778782844543%
train set accuracy: 99.95707869529724%
